In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

## Loading and preparing label maps

In [4]:
with open('../../temp/all_labels_names_300.pkl', 'rb') as fin:
    mid2name_all = pickle.load(fin)

In [5]:
with open('../../temp/all_labels_300+.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [6]:
with open('../../results/det_results_concat_20.pkl', 'rb') as fin:
    det_results_concat = pickle.load(fin)

In [7]:
with open('../../results/det_results_merged_20.pkl', 'rb') as fin:
    det_results_merged = pickle.load(fin)

In [8]:
with open('../../results/det_results_postproc_20.pkl', 'rb') as fin:
    det_results_postproc = pickle.load(fin)

## Choosing examples for each class to visualize

In [9]:
label_to_img = {key: [] for key in all_labels}
for key, val in det_results_concat.items():
    for det in val:
        if det['score'] >= 0.1:
            label_to_img[det['label']].append(key)

In [10]:
sort_idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [11]:
for i in sort_idx:
    print(all_labels[i], mid2name_all[all_labels[i]], len(label_to_img[all_labels[i]]))

/m/01g317 Person 6923
/m/04yx4 Man 2117
/m/0dzct Human face 1884
/m/09j2d Clothing 1505
/m/03bt1vf Woman 979
/m/0k4j Car 596
/m/07yv9 Vehicle 483
/m/0cgh4 Building 368
/m/01xyhv Suit 353
/m/01rkbr Tie 300
/m/07j7r Tree 297
/m/0cmf2 Airplane 290
/m/01n5jq Poster 254
/m/01prls Land vehicle 174
/m/019jd Boat 154
/m/07r04 Truck 151
/m/05s2s Plant 146
/m/07jdr Train 136
/m/04dr76w Bottle 134
/m/083wq Wheel 131
/m/080hkjn Handbag 130
/m/04_tb Map 126
/m/0bt9lr Dog 107
/m/01mzpv Chair 107
/m/0jyfg Glasses 107
/m/09j5n Footwear 104
/m/050k8 Mobile phone 100
/m/0fly7 Jeans 99
/m/012t_z Businessperson 95
/m/07c52 Television 93
/m/04bcr3 Table 89
/m/0zvk5 Helmet 87
/m/0k65p Hand 86
/m/032b3c Jacket 83
/m/03q69 Hair 80
/m/01d40f Dress 79
/m/03120 Flag 73
/m/07cmd Tank 69
/m/0dwx7 Logo 68
/m/03jm5 House 67
/m/01940j Backpack 66
/m/03fp41 Houseplant 63
/m/01xr66 Spokesperson 63
/m/0d4v4 Window 63
/m/015p6 Bird 62
/m/01yrx Cat 56
/m/02_41 Fire 55
/m/0h9mv Tire 53
/m/02dl1y Hat 52
/m/07s6nbt Text 51
/

In [12]:
#select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3', '/m/0dwx7']


In [10]:
with open('../../../wsod/metadata/ont_300/300-classes.csv', 'r') as fin:
    select_labels = [item.strip() for item in fin]


## Visualizing Results

In [11]:
mid2name_extended = {}
extended_classes = set()
for key in all_labels:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('pascal','PA'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J'), ('pascal/J','PA/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_all[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [12]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [13]:
result_dict = {
    'Not Merged': det_results_concat,
    'Merged': det_results_merged,
    'Post-Processed': det_results_postproc,
}

In [14]:
path_prefix = '../../../../data/eval_m9/jpg/' 

In [15]:
model_abv_dict = {
    'coco': 'CO',
    'pascal': 'PA',
    'oi': 'OI',
    'ws': 'WS',
    'coco/J': 'CO/J',
    'pascal/J': 'PA/J',
    'oi/J': 'OI/J',
    'ws/J': 'WS/J',
}

In [16]:
def show(imgid, source, thresh):
    filename = path_prefix + imgid + '.jpg.ldcc'
    print(imgid)
    with open(filename, 'rb') as fin:
        _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    
    detections = result_dict[source][imgid]
    boxes = np.asarray([det['bbox_normalized'] for det in detections])[:,[1,0,3,2]]
    scores = [det['score'] for det in detections]
    label_idx = [mid2idx_extended[det['label'] + '/' + det['model']] for det in detections]
    
    # print([(mid2name_extended[extended_classes[item]], boxes[i]) for i, item in enumerate(label_idx) if scores[i] > thresh])
    
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      boxes,
      label_idx,
      scores,
      category_index_extended,
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      max_boxes_to_draw=1000,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [17]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        imgid=fixed(imgid),
        source=widgets.RadioButtons(options=['Post-Processed', 'Merged', 'Not Merged'], value='Post-Processed'), 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, continuous_update=False), 
    )
    w.children[-1].layout.height = '600px'
    display(w)
    

Air force


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

Aircraft


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

Aircraft cabin


...

interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

Rescue


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…

Rebellion


interactive(children=(RadioButtons(description='source', options=('Post-Processed', 'Merged', 'Not Merged'), v…